## Modelos de languaje natural y HuggingFaces

En este cuaderno, exploraremos varios modelos avanzados de lenguaje natural (NLP), incluidos GPT-1, GPT-2, GPT-3, BART, ELECTRA, XLNet, Transformer-XL, y métodos de difusión orientados a LLM. Cada sección incluye una breve explicación teórica y ejemplos de implementación en Python.

#### GPT-1

Generative Pre-trained Transformer 1 (GPT-1) es un modelo de lenguaje introducido por OpenAI en 2018. Utiliza el enfoque de aprendizaje no supervisado, preentrenando un modelo Transformer en una gran cantidad de texto y luego ajustándolo para tareas específicas de NLP.



In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = 'gpt2'  # Utilizamos GPT-2 como proxy para GPT-1
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

def generate_text(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "El aprendizaje automático es"
generated_text = generate_text(prompt)
print(generated_text)


**GPT-2**

GPT-2 es una versión mejorada de GPT-1 con más parámetros y entrenada en una mayor cantidad de datos. Introducida por OpenAI en 2019, GPT-2 demostró capacidades avanzadas en la generación de texto coherente y relevante.

**Implementación y ejemplos**

In [ ]:
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

prompt = "La inteligencia artificial tiene el potencial de"
generated_text = generate_text(prompt)
print(generated_text)


#### GPT-3

GPT-3, también de OpenAI, es una versión aún más avanzada con 175 mil millones de parámetros. Lanzado en 2020, GPT-3 puede realizar una amplia gama de tareas de NLP con mínima o ninguna modificación específica de la tarea.



In [ ]:
from transformers import GPT3Tokenizer, GPT3Model  # Asumiendo que existe una clase GPT-3 en transformers

# GPT-3 no está disponible gratuitamente en transformers, se usa a través de la API de OpenAI

api_key = 'your_openai_api_key'
openai.api_key = api_key

def generate_text_gpt3(prompt, max_length=50):
    response = openai.Completion.create(
        engine="davinci",
        prompt=prompt,
        max_tokens=max_length
    )
    return response.choices[0].text.strip()

prompt = "Los avances en la inteligencia artificial incluyen"
generated_text = generate_text_gpt3(prompt)
print(generated_text)


**BART**

BART (Bidirectional and Auto-Regressive Transformers) es un modelo de Facebook AI que combina los enfoques de modelos auto-regresivos y auto-encoders. Es eficaz en tareas de generación y comprensión de lenguaje.

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

model_name = 'facebook/bart-large'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

def generate_text_bart(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    outputs = model.generate(inputs.input_ids, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "La ciencia de datos es"
generated_text = generate_text_bart(prompt)
print(generated_text)


**ELECTRA**

ELECTRA (Efficiently Learning an Encoder that Classifies Token Replacements Accurately) es un modelo de Google Research que introduce un nuevo enfoque de preentrenamiento más eficiente que BERT.

In [ ]:
from transformers import ElectraTokenizer, ElectraForPreTraining

model_name = 'google/electra-small-discriminator'
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraForPreTraining.from_pretrained(model_name).to(device)

def predict_masked_word(prompt):
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    discriminator_outputs = model(**inputs)
    predictions = discriminator_outputs.logits.argmax(dim=-1)
    return tokenizer.decode(predictions[0], skip_special_tokens=True)

prompt = "El aprendizaje profundo es una rama de la [MASK]"
predicted_word = predict_masked_word(prompt)
print(predicted_word)


**XLNet**

XLNet es un modelo de Google AI que combina las ventajas de BERT y los modelos auto-regresivos. Introduce una permutación de tokens en su entrenamiento, mejorando la capacidad de capturar dependencias bidireccionales.

In [ ]:
from transformers import XLNetTokenizer, XLNetLMHeadModel

model_name = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(model_name)
model = XLNetLMHeadModel.from_pretrained(model_name).to(device)

prompt = "El procesamiento de lenguaje natural"
generated_text = generate_text(prompt)
print(generated_text)


**Transformer-XL**

Transformer-XL es una extensión del modelo Transformer que introduce la memoria segmentada recurrente, permitiendo a los modelos manejar dependencias a largo plazo de manera más efectiva.


In [ ]:
from transformers import TransfoXLTokenizer, TransfoXLLMHeadModel

model_name = 'transfo-xl-wt103'
tokenizer = TransfoXLTokenizer.from_pretrained(model_name)
model = TransfoXLLMHeadModel.from_pretrained(model_name).to(device)

prompt = "La inteligencia artificial"
generated_text = generate_text(prompt)
print(generated_text)


### Métodos de difusión orientados a LLM

Los métodos de difusión son una clase de modelos generativos que se inspiran en procesos de difusión física, como la propagación de calor o la difusión de partículas. En el contexto de LLM (Modelos de Lenguaje a Gran Escala), los métodos de difusión pueden ser utilizados para mejorar la calidad y coherencia de la generación de texto.

Los modelos de difusión funcionan mediante un proceso de denoising (eliminación de ruido) iterativo. Inicialmente, se añade ruido a una entrada estructurada, y luego, el modelo aprende a revertir este proceso para recuperar la estructura original. Este enfoque puede ser utilizado para tareas de generación de texto donde se busca mantener una coherencia semántica y estilística a lo largo de la secuencia generada.


A continuación, implementamos un modelo de difusión simple para generación de texto utilizando una arquitectura basada en Transformers. Primero, definimos las funciones para el proceso de adición de ruido y denoising:


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Parámetros del modelo
model_name = 'gpt2'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo y el tokenizer
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Función para añadir ruido al texto
def add_noise(tokens, noise_level=0.1):
    noisy_tokens = tokens.clone()
    mask = torch.rand(tokens.shape) < noise_level
    noisy_tokens[mask] = tokenizer.pad_token_id
    return noisy_tokens

# Función de denoising
class DenoisingModel(nn.Module):
    def __init__(self, base_model):
        super(DenoisingModel, self).__init__()
        self.base_model = base_model
    
    def forward(self, input_ids, attention_mask=None):
        outputs = self.base_model(input_ids, attention_mask=attention_mask)
        return outputs.logits

# Entrenamiento del modelo de denoising
def train_denoising_model(model, tokenizer, texts, epochs=5, noise_level=0.1):
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        epoch_loss = 0
        for text in texts:
            inputs = tokenizer(text, return_tensors='pt').input_ids.to(device)
            noisy_inputs = add_noise(inputs, noise_level)
            outputs = model(noisy_inputs)
            loss = criterion(outputs.view(-1, tokenizer.vocab_size), inputs.view(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(texts)}')

# Datos de ejemplo y entrenamiento
texts = [
    "La inteligencia artificial está revolucionando el mundo.",
    "El aprendizaje automático es una subdisciplina de la inteligencia artificial.",
    "Los modelos de lenguaje a gran escala tienen un impacto significativo en la tecnología moderna."
]

denoising_model = DenoisingModel(model)
train_denoising_model(denoising_model, tokenizer, texts)

# Generación de texto con el modelo de denoising
def generate_text_with_denoising(model, tokenizer, prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
    noisy_inputs = add_noise(inputs)
    denoised_outputs = model(noisy_inputs)
    generated_text = tokenizer.decode(denoised_outputs.argmax(dim=-1).squeeze(), skip_special_tokens=True)
    return generated_text

prompt = "La tecnología moderna"
generated_text = generate_text_with_denoising(denoising_model, tokenizer, prompt)
print(generated_text)


####  GPT-1

**Ejercicio 1: Generación de texto con GPT-1**

Utilizando el modelo GPT-1 (representado por GPT-2 en Hugging Face), genera un texto de al menos 100 palabras a partir de un prompt dado. Experimenta con diferentes prompts y observa cómo cambia la coherencia del texto generado.

```python
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Prompt de ejemplo
prompt = "En el futuro, la inteligencia artificial"
generated_text = generate_text(prompt)
print(generated_text)


In [ ]:
## Tu respuesta

**Ejercicio 2:**  Ajuste fino de GPT-1 para una tarea específica 

Encuentra un conjunto de datos pequeño y ajusta finamente GPT-1 para una tarea específica (por ejemplo, generación de noticias, diálogos, etc.). Describe los pasos y resultados obtenidos.



####  Este es un ejemplo de plantilla para el ajuste fino

```
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset 
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
 
def preprocess(data):
    return tokenizer(data['text'], truncation=True, padding='max_length', max_length=512)

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
dataset = dataset.map(preprocess, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

trainer.train()
``` 

In [ ]:
## Tu respuesta

**Ejercicio 3:** Exploración de parámetros de generación

Experimenta con diferentes parámetros de generación (como `max_length`, `temperature`, `top_k`, `top_p`) y analiza cómo afectan la calidad y diversidad del texto generado por GPT-2.

In [ ]:
def generate_text_with_params(prompt, max_length=100, temperature=1.0, top_k=50, top_p=0.95):
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    outputs = model.generate(inputs.input_ids, max_length=max_length, temperature=temperature, top_k=top_k, top_p=top_p, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Prueba diferentes configuraciones de parámetros
prompt = "La revolución tecnológica"
generated_text = generate_text_with_params(prompt, max_length=100, temperature=0.7, top_k=30, top_p=0.8)
print(generated_text)


In [ ]:
## Tu respuesta

**Ejercicio 4:** Evaluación de GPT-2 en una tarea de resumen de textos

Utiliza GPT-2 para generar resúmenes de textos largos. Evalúa la calidad de los resúmenes generados comparándolos con resúmenes humanos.

####  Este es un ejemplo de plantilla para la generación de resúmenes

```
texts = [
    "El aprendizaje automático es una rama de la inteligencia artificial que se centra en el desarrollo de algoritmos que permiten a las máquinas aprender y mejorar a partir de la experiencia. ...",
    "La inteligencia artificial ha avanzado significativamente en los últimos años, permitiendo aplicaciones en diversos campos como la medicina, el transporte y la robótica. ..."
]

for text in texts:
    summary = generate_text_with_params(text, max_length=50, temperature=0.7, top_k=30, top_p=0.8)
    print(f"Texto original: {text}")
    print(f"Resumen generado: {summary}")
``` 

In [ ]:
## Tu respuesta

**Ejercicio 5:** Uso de la API de OpenAI para GPT-3

Crea una cuenta en OpenAI y utiliza la API de GPT-3 para generar texto. Prueba diferentes prompts y analiza los resultados.

In [ ]:
import openai

api_key = 'your_openai_api_key'
openai.api_key = api_key

def generate_text_gpt3(prompt, max_length=50):
    response = openai.Completion.create(
        engine="davinci",
        prompt=prompt,
        max_tokens=max_length
    )
    return response.choices[0].text.strip()

# Prompt de ejemplo
prompt = "La inteligencia artificial en la medicina"
generated_text = generate_text_gpt3(prompt)
print(generated_text)


In [ ]:
## Tu respuesta

**Ejercicio 6:** Generación de resúmenes con BART

Utiliza BART para generar resúmenes de artículos de noticias. Compara la calidad de los resúmenes generados con los de otros modelos como GPT-2.


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

def generate_summary(text, max_length=50):
    inputs = tokenizer(text, return_tensors='pt', max_length=1024, truncation=True).to(device)
    summary_ids = model.generate(inputs.input_ids, max_length=max_length, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Texto de ejemplo
text = "El cambio climático es uno de los mayores desafíos de nuestro tiempo. Sus efectos se sienten en todo el mundo, con fenómenos meteorológicos extremos cada vez más frecuentes. ..."
summary = generate_summary(text)
print(summary)


In [ ]:
## Tu respuesta

**Ejercicio 7:** Traducción automática con BART

Utiliza BART para realizar traducción automática entre dos idiomas. Evalúa la calidad de las traducciones generadas.



In [ ]:
## Tu respuesta

**Ejercicio 8:** Clasificación de texto con ELECTRA

Ajusta finamente ELECTRA para una tarea de clasificación de texto, como la detección de spam en correos electrónicos. Evalúa el rendimiento del modelo.

In [ ]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

model_name = 'google/electra-small-discriminator'
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

# Cargar datos de ejemplo (detección de spam)
dataset = load_dataset('sms_spam', split='train')

# Preprocesamiento
def preprocess(data):
    return tokenizer(data['sms'], truncation=True, padding='max_length', max_length=128)

dataset = dataset.map(preprocess, batched=True)
dataset = dataset.rename_column("label", "labels")

# Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# Entrenar el modelo
trainer.train()

# Evaluación del modelo
trainer.evaluate()


In [ ]:
## Tu respuesta

**Ejercicio 9:** Generación de texto con ELECTRA

Utiliza ELECTRA para generar texto a partir de un prompt dado. Compara la calidad del texto generado con el de otros modelos como GPT-2 y BART.

####  Este es un ejemplo de plantilla para la generación de texto con ELECTRA (necesita ajuste fino)

```
prompt = "La inteligencia artificial"
generated_text = generate_text_with_params(prompt, max_length=50, temperature=0.7, top_k=30, top_p=0.8)
print(generated_text)
``` 

In [ ]:
## Tu respuesta

**Ejercicio 10:** Completar Texto con XLNet

Utiliza XLNet para completar un texto dado. Experimenta con diferentes prompts y analiza la coherencia del texto completado.

In [ ]:
from transformers import XLNetTokenizer, XLNetLMHeadModel

model_name = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(model_name)
model = XLNetLMHeadModel.from_pretrained(model_name).to(device)

def complete_text(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Prompt de ejemplo
prompt = "El futuro de la inteligencia artificial"
completed_text = complete_text(prompt)
print(completed_text)


In [ ]:
## Tu respuesta

**Ejercicio 11:** Ajuste fino de XLNet para clasificación de sentimientos

Ajusta finamente XLNet para una tarea de clasificación de sentimientos en un conjunto de datos de revisiones de productos. Evalúa el rendimiento del modelo.


####  Este es un ejemplo de plantilla para la clasificación de sentimientos

```
from transformers import XLNetForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

model_name = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(model_name)
model = XLNetForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
dataset = load_dataset('imdb', split='train')

def preprocess(data):
    return tokenizer(data['text'], truncation=True, padding='max_length', max_length=128)

dataset = dataset.map(preprocess, batched=True)
dataset = dataset.rename_column("label", "labels")

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

trainer.train()
trainer.evaluate()
``` 

In [ ]:
## Tu respuesta

#### Ejercicio 12: Generación de texto a largo plazo con Transformer-XL

Utiliza Transformer-XL para generar texto coherente a largo plazo a partir de un prompt dado. Compara la coherencia del texto generado con el de otros modelos.

In [ ]:
from transformers import TransfoXLTokenizer, TransfoXLLMHeadModel

model_name = 'transfo-xl-wt103'
tokenizer = TransfoXLTokenizer.from_pretrained(model_name)
model = TransfoXLLMHeadModel.from_pretrained(model_name).to(device)

def generate_long_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Prompt de ejemplo
prompt = "La historia de la inteligencia artificial comienza"
long_text = generate_long_text(prompt)
print(long_text)


In [ ]:
## Tu respuesta

**Ejercicio13:** Implementación de un modelo de difusión para generación de texto
    
Implementa un modelo de difusión simple para la generación de texto y evalúa su rendimiento en comparación con otros modelos.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name = 'gpt2'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

def add_noise(tokens, noise_level=0.1):
    noisy_tokens = tokens.clone()
    mask = torch.rand(tokens.shape) < noise_level
    noisy_tokens[mask] = tokenizer.pad_token_id
    return noisy_tokens

class DenoisingModel(nn.Module):
    def __init__(self, base_model):
        super(DenoisingModel, self).__init__()
        self.base_model = base_model
    
    def forward(self, input_ids, attention_mask=None):
        outputs = self.base_model(input_ids, attention_mask=attention_mask)
        return outputs.logits

def train_denoising_model(model, tokenizer, texts, epochs=5, noise_level=0.1):
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        epoch_loss = 0
        for text in texts:
            inputs = tokenizer(text, return_tensors='pt').input_ids.to(device)
            noisy_inputs = add_noise(inputs, noise_level)
            outputs = model(noisy_inputs)
            loss = criterion(outputs.view(-1, tokenizer.vocab_size), inputs.view(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(texts)}')

texts = [
    "La inteligencia artificial está revolucionando el mundo.",
    "El aprendizaje automático es una subdisciplina de la inteligencia artificial.",
    "Los modelos de lenguaje a gran escala tienen un impacto significativo en la tecnología moderna."
]

denoising_model = DenoisingModel(model)
train_denoising_model(denoising_model, tokenizer, texts)

def generate_text_with_denoising(model, tokenizer, prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
    noisy_inputs = add_noise(inputs)
    denoised_outputs = model(noisy_inputs)
    generated_text = tokenizer.decode(denoised_outputs.argmax(dim=-1).squeeze(), skip_special_tokens=True)
    return generated_text

prompt = "La tecnología moderna"
generated_text = generate_text_with_denoising(denoising_model, tokenizer, prompt)
print(generated_text)


In [ ]:
## Tu respuesta

**Ejercicio 14:** Comparación de métodos de difusión

Implementa varios métodos de difusión y compáralos en términos de calidad y coherencia del texto generado. Describe los resultados obtenidos.

In [ ]:
## Tus respuestas